In [5]:
# importing libraries
import cv2
import numpy as np
import math

In [6]:
def mid_line_equation(x_inter, y_inter, x_l, y_l, x_r, y_r):
    x_mid = (x_l + x_r)/2
    y_mid = (y_l + y_r)/2
    a_mid = (y_mid - y_inter)/(x_mid-x_inter+0.00001)
    b_mid = y_mid - a_mid * x_mid

    theta = np.arctan(-1/a_mid)
    rho = np.sin(theta) * b_mid

    print('Mid line equation:')

    print('y = {a}x + {b}'.format(a = a_mid, b = b_mid))
    print('theta = {th}, rho = {r}\n'.format(th = theta, r = rho))
    return (int(y_inter), int(x_inter), int(y_mid), int(x_mid), int(x_inter), int(y_inter), a_mid, b_mid)


def detect_mid_line(ptl1, ptl2, ptr1, ptr2):
    #calculate the center line
    y_l1 = ptl1[0]
    x_l1 = ptl1[1]
    y_l2 = ptl2[0]
    x_l2 = ptl2[1]

    a_l = (y_l1-y_l2)/(x_l1-x_l2+0.000001)
    b_l = y_l1 - x_l1 * a_l

    y_r1 = ptr1[0]
    x_r1 = ptr1[1]
    y_r2 = ptr2[0]
    x_r2 = ptr2[1]

    a_r = (y_r1-y_r2)/(x_r1-x_r2+0.0000001)
    b_r = y_r1 - x_r1 * a_r

    #get intersect point
    x_inter = (b_r-b_l)/(a_l-a_r+0.00001)
    y_inter = x_inter*a_l + b_l

    #get mid_point left
    x_ml = x_inter + 10000
    y_ml = x_ml * a_l + b_l
    d_2 = np.square(y_inter - y_ml) + np.square(x_inter - x_ml)

    #get mid_point right
    #delta = b^2 - 4ac
    a_mr = np.square(a_r) + 1
    b_mr = -2*(x_inter+y_inter*a_r-a_r*b_r)
    c_mr = -(d_2 - np.square(x_inter) - np.square(y_inter) + 2*y_inter*b_r - np.square(b_r))
    delta = np.square(b_mr) - 4*a_mr*c_mr

    #get x_mid_right and y_mid_right
    x_mr1 = (-b_mr - np.sqrt(delta)) / (2*a_mr+0.000001)
    x_mr2 = (-b_mr + np.sqrt(delta)) / (2*a_mr+0.000001)

    y_mr1 = a_r * x_mr1 + b_r
    y_mr2 = a_r * x_mr2 + b_r

    #return mid_line
    if x_mr1 > x_inter:
        return mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr1, y_mr1)
    else:
        return mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr2, y_mr2)

In [7]:
def lane_making(img):
    #import image
    #cover to hsv
    blur_img = cv2.GaussianBlur(img,(5, 5), 2)
    img_gray = cv2.cvtColor(blur_img, cv2.COLOR_BGR2GRAY)

    #blur
    line_mask = np.ones((len(img), len(img[0])))
    line_mask[:int(len(img)/2),:] = 0
    line_mask[-int(len(img)/15):, :] = 0

    line_img = line_mask*img_gray
    blur_img = cv2.GaussianBlur(line_img,(5, 5), 5)

    kernel1 = np.ones((5, 5), np.uint8)
    erosion = cv2.erode(blur_img, kernel1, iterations=8)
    dilation = cv2.dilate(erosion, kernel1, iterations=1)

    bw = cv2.threshold(dilation,210,255,cv2.THRESH_BINARY_INV)[1]

    #find edges
    t_lower = 0  # Lower Threshold
    t_upper = 255 # Upper threshold

    # Applying the Canny Edge filter
    edges = cv2.Canny(np.uint8(bw), t_lower, t_upper)
    lines = cv2.HoughLines(edges, 1, np.pi / 50, 50, None, 0, 0)

    if lines is None:
        cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
        cv2.imshow('Frame', img)
        return None, None, None, None
    else:
        len_line = len(lines)
        get_min = False
        get_max = False

        if len(lines) < 3:
            cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
            cv2.imshow('Frame', img)
            return None, None, None, None
        else:
            #draw lines
            min_pt = [len(img[0]), 0, 0]
            max_pt = [0, 0, 0]

            #get 2 main lines
            for i in range(0, len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
                pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

                if abs(pt1[1]-pt2[1]) >30:
                    if min_pt[0] > rho:
                        min_pt[0] = x0
                        min_pt[1] = pt1
                        min_pt[2] = pt2
                        get_min = True


                    if max_pt[0] < rho:
                        max_pt[0] = x0
                        max_pt[1] = pt1
                        max_pt[2] = pt2
                        get_max = True
                else:
                    len_line -= 1

            #get center line

            if len_line > 2 and get_min and get_max:
                #get center

                center = detect_mid_line(min_pt[1], min_pt[2], max_pt[1], max_pt[2])
                x_inter = center[4]
                y_inter = center[5]
                a_mid = center[6]
                b_mid = center[7]

                if x_inter < 0.5 * len(img):
                    cv2.putText(img, ' y = {a}x + {b}'.format(a = round(a_mid,3), b = round(b_mid,3)), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                    cv2.line(img, min_pt[1], min_pt[2], (100,0,255), 3, cv2.LINE_AA)
                    cv2.line(img, max_pt[1], max_pt[2], (100,0,255), 3, cv2.LINE_AA)
                    cv2.line(img, (center[0], center[1]), (center[2], center[3]), (255,100,0), 3, cv2.LINE_AA)
                    cv2.imshow('Frame', img)
                    return x_inter, y_inter, a_mid, b_mid
                else:
                    cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                    cv2.imshow('Frame', img)
                    return None, None, None, None
            else:
                cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                cv2.imshow('Frame', img)
                return None, None, None, None


In [9]:
# Create a VideoCapture object and read from input fileqqqqq
cap = cv2.VideoCapture('../../data/line_trace/congthanh_solution.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        x_inter, y_inter, a_mid, b_mid = lane_making(frame)

    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

Mid line equation:
y = -0.7269452922160637x + 1175.6435972802792
theta = 0.942214233963709, rho = 950.9334882307966

Mid line equation:
y = -0.5096917548695762x + 1407.010100829657
theta = 1.0994254090273996, rho = 1253.5708383741144

Mid line equation:
y = -0.4329315684959033x + 1385.0784806310312
theta = 1.1622267965786224, rho = 1271.0727857065067

Mid line equation:
y = -0.2905903186710115x + 1273.5236230420446
theta = 1.2879944666536396, rho = 1222.935898945965

Mid line equation:
y = -0.22338674295436656x + 1181.0470258446549
theta = 1.3510179352139298, rho = 1152.6378689904766

Mid line equation:
y = -0.25666830556813913x + 1188.9109963217213
theta = 1.3195515251640653, rho = 1151.5835926928908

Mid line equation:
y = -0.25698017591713995x + 1181.259025313801
theta = 1.3192589525250344, rho = 1144.0858956930597

Mid line equation:
y = -0.19111674237771434x + 921.8313690936741
theta = 1.3819567687605065, rho = 905.4437273242377

Mid line equation:
y = -0.19108417001169828x + 885.

/tmp/ipykernel_59526/2961138534.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  theta = np.arctan(-1/a_mid)


Mid line equation:
y = 0.9395474273071976x + -789.9716712131631
theta = -0.8165564672981784, rho = 575.7252304384509

Mid line equation:
y = 0.9370255963781831x + -7126.435032769572
theta = -0.8178975948535836, rho = 5200.230263072043

Mid line equation:
y = 0.9386248032826345x + -140.51372091902886
theta = -0.8170467337952966, rho = 102.45246910861032

Mid line equation:
y = -0.32456601745905533x + 220.43450473464372
theta = 1.2569570001368002, rho = 209.66745500108095

Mid line equation:
y = 0.0628052798348162x + 454.1695564721168
theta = -1.508073430612161, rho = -453.27646108974307

Mid line equation:
y = -0.4322273270268779x + 340.0727407653885
theta = 1.1628200293659416, rho = 312.16147835594074

Mid line equation:
y = -0.4321181433144742x + 340.66725745737676
theta = 1.1629120298472901, rho = 312.7196339703029

Mid line equation:
y = -0.5496575335102619x + 291.10787090232316
theta = 1.0682160840156902, rho = 255.1102306970283

Mid line equation:
y = -0.509121803003874x + 381.657